In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from scipy.optimize import minimize
from scipy.optimize import linprog
from joblib import Parallel, delayed
import os
import tempfile
from RiskMeasureEst import *
import random
from Env import Env
from Agent import *
from tqdm import tqdm

In [2]:
gamma = 0.9
aph = 0.50
B = 5
p1 = 0.75
O = 1

In [3]:
p = np.zeros((B,2,B))

p[0,0,1] = p1
p[0,0,0] = 1-p1
p[0,1,1] = 1-p1
p[0,1,0] = p1

p[B-1,0,B-1] = p1
p[B-1,0,0] = 1-p1
p[B-1,1,B-1] = 1-p1
p[B-1,1,0] = p1

for i in range(1,B-1):
    p[i,0,i+1] = p1
    p[i,0,0] = 1-p1
    p[i,1,i+1] = 1-p1
    p[i,1,0] = p1

In [4]:
r = np.zeros((B,2,B))

r[0,0,0] = 2
r[0,1,0] = 2

r[B-1,0,B-1] = 10
r[B-1,1,B-1] = 10

for i in range(1,B):
    r[i,0,0] = 2
    r[i,1,0] = 2

In [5]:
Priors = []

alpha0 =  np.zeros((B,2,B))
alpha0 += 1/B
alpha0 *= O

Priors.append(alpha0)

alpha0 =  np.zeros((B,2,B))
alpha0 += 1e-10
alpha0[0,0,1] = 1/2
alpha0[0,0,0] = 1/2
alpha0[0,1,1] = 1/2
alpha0[0,1,0] = 1/2

alpha0[B-1,0,B-1] = 1/2
alpha0[B-1,0,0] = 1/2
alpha0[B-1,1,B-1] = 1/2
alpha0[B-1,1,0] = 1/2


for i in range(1,B-1):
    alpha0[i,0,i+1] = 1/2
    alpha0[i,0,0] = 1/2
    alpha0[i,1,i+1] = 1/2
    alpha0[i,1,0] = 1/2
alpha0 *= O

Priors.append(alpha0)

alpha0 =  np.zeros((B,2,B))
alpha0 += 1e-10
alpha0[0,0,1] = 2/3
alpha0[0,0,0] = 1/3
alpha0[0,1,1] = 1/3
alpha0[0,1,0] = 2/3

alpha0[B-1,0,B-1] = 2/3
alpha0[B-1,0,0] = 1/3
alpha0[B-1,1,B-1] = 1/3
alpha0[B-1,1,0] = 2/3


for i in range(1,B-1):
    alpha0[i,0,i+1] = 2/3
    alpha0[i,0,0] = 1/3
    alpha0[i,1,i+1] = 1/3
    alpha0[i,1,0] = 2/3
    
alpha0 *= O

Priors.append(alpha0)

In [6]:
def train(p,r,B,A,alpha0,
                  N=10,
                  aph=0.5, 
                  kappa = None,
                  risk_type='CVaR',
                  aph_beta = 0.2,
                  beta_type='Mean',
                  Prior_name='1',
                  task='CP'):
    
    action_array = np.zeros((B,1000,20))
    
    for i in range(20):
        print('step,',i)
        env = Env(p,-r,B=B,A=A)
        agent = Agent(env,alpha0.copy(),N=N,
                      aph=aph, \
                      kappa = kappa,\
                      risk_type=risk_type,\
                      aph_beta = aph_beta,\
                      beta_type=beta_type)

        for j in tqdm(range(1000)):
            if j%100 == 0:
                agent.DP()

            s0idx,aidx,s1idx, cost = agent.env.step(agent.selectAction(env.sidx))
            agent.updateAlpha(s0idx,aidx,s1idx)
            
            action_array[:,j,i] = agent.Q.argmin(axis=1)
            
    np.save('res/rho_{}_beta_{}_Prior_{}_task_{}'.format(risk_type,beta_type,Prior_name,task),\
            action_array)

In [7]:
train(p,r,5,2,Priors[0],
                  N=10,
                  aph=0.5, 
                  kappa = None,
                  risk_type='CVaR',
                  aph_beta = 0.2,
                  beta_type='Mean',
                  Prior_name='1',
                  task = 'CP')

step, 0


100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:13<00:00, 72.73it/s]


step, 1


100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:19<00:00, 52.57it/s]


step, 2


...

100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:15<00:00, 62.60it/s]


step, 18


100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:22<00:00, 43.61it/s]


step, 19


100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:17<00:00, 58.38it/s]


In [8]:
train(p,r,5,2,Priors[1],
                  N=10,
                  aph=0.5, 
                  kappa = None,
                  risk_type='CVaR',
                  aph_beta = 0.2,
                  beta_type='Mean',
                  Prior_name='2')

step, 0


100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:18<00:00, 54.67it/s]


step, 1


100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:15<00:00, 62.59it/s]


step, 2


...

100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:18<00:00, 54.92it/s]


step, 18


100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:21<00:00, 46.94it/s]


step, 19


100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:15<00:00, 66.36it/s]


In [9]:
train(p,r,5,2,Priors[1],
                  N=10,
                  aph=0.5, 
                  kappa = None,
                  risk_type='Mean',
                  aph_beta = 0.2,
                  beta_type='Mean',
                  Prior_name='2')

step, 0


100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:10<00:00, 96.90it/s]


step, 1


100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:11<00:00, 84.37it/s]


step, 2


...

100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:14<00:00, 70.94it/s]


step, 18


100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:10<00:00, 99.09it/s]


step, 19


100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:13<00:00, 74.99it/s]


In [10]:
train(p,r,5,2,Priors[1],
                  N=10,
                  aph=0.5, 
                  kappa = None,
                  risk_type='Mean',
                  aph_beta = 0.2,
                  beta_type='CVaR',
                  Prior_name='2')

step, 0


100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:20<00:00, 48.77it/s]


step, 1


100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:12<00:00, 78.95it/s]


step, 2


...

100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:12<00:00, 81.17it/s]


step, 18


100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:10<00:00, 92.47it/s]


step, 19


100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:12<00:00, 79.75it/s]


In [11]:
train(p,r,5,2,Priors[1],
                  N=10,
                  aph=0.5, 
                  kappa = None,
                  risk_type='CVaR',
                  aph_beta = 0.2,
                  beta_type='CVaR',
                  Prior_name='2')

step, 0


100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:21<00:00, 45.88it/s]


step, 1


100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:21<00:00, 46.88it/s]


step, 2


...

100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:17<00:00, 58.23it/s]


step, 18


100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:23<00:00, 41.99it/s]


step, 19


100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:19<00:00, 51.82it/s]


In [12]:
train(p,r,5,2,Priors[2],
                  N=10,
                  aph=0.5, 
                  kappa = None,
                  risk_type='CVaR',
                  aph_beta = 0.2,
                  beta_type='Mean',
                  Prior_name='3')

step, 0


100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:18<00:00, 53.86it/s]


step, 1


100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:24<00:00, 40.29it/s]


step, 2


...

100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:19<00:00, 52.48it/s]


step, 18


100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:14<00:00, 66.70it/s]


step, 19


100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:20<00:00, 49.12it/s]
